In [1]:
import pandas as pd
import os
import datetime
import arcpy

C:\Users\htran\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
gdb_path = "G:/usgrids2020/final_gdbs"
gdb_list = os.listdir(gdb_path)
gdb_list.sort()

arcpy.env.workspace = os.path.join(gdb_path, gdb_list[8])

fc = "usade_joined"
fcInMem = arcpy.management.CopyFeatures(fc, "in_memory/usade_inMem")



In [6]:
arcpy.management.CalculateGeometryAttributes(
    in_features=fcInMem,
    geometry_property="AREA AREA_GEODESIC",
    length_unit="",
    area_unit="SQUARE_METERS",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

<Result 'in_memory\\usade_inMem'>

In [8]:
arcpy.management.CalculateGeometryAttributes(fcInMem, "AREA AREA_GEODESIC", "", "SQUARE_METERS")
for field in arcpy.ListFields(fcInMem):
    print(field.name)

OBJECTID
Shape
STATEFP20
COUNTYFP20
TRACTCE20
BLOCKCE20
GEOID20
NAME20
MTFCC20
UR20
UACE20
UATYPE20
FUNCSTAT20
ALAND20
AWATER20
INTPTLAT20
INTPTLON20
HOUSING20
POP20
USCID
ISO
UCADMIN0
NAME0
UCADMIN1
NAME1
UCADMIN2
NAME2
UCADMIN3
NAME3
UCADMIN4
NAME4
UCADMIN5
NAME5
CENSUS_YEA
POP_CONTEX
ATOTPOPMT
ATOTPOPFT
ATOTPOPBT
A000_004MT
A005_009MT
A010_014MT
A015_019MT
A020_024MT
A025_029MT
A030_034MT
A035_039MT
A040_044MT
A045_049MT
A050_054MT
A055_059MT
A060_064MT
A065_069MT
A070_074MT
A075_079MT
A080_084MT
A085plusMT
A000_004FT
A005_009FT
A010_014FT
A015_019FT
A020_024FT
A025_029FT
A030_034FT
A035_039FT
A040_044FT
A045_049FT
A050_054FT
A055_059FT
A060_064FT
A065_069FT
A070_074FT
A075_079FT
A080_084FT
A085plusFT
A000_004BT
A005_009BT
A010_014BT
A015_019BT
A020_024BT
A025_029BT
A030_034BT
A035_039BT
A040_044BT
A045_049BT
A050_054BT
A055_059BT
A060_064BT
A065_069BT
A070_074BT
A075_079BT
A080_084BT
A085plusBT
union_uid
TOTALPOP
AUND1
A1TO4
A5TO17
A18TO24
AUND25
A25OV
A25TO64
A65TO79
A80OV
WOMCHIL

In [9]:
arcpy.management.CalculateField(fcInMem, "AREA", "!shape.area!")

<Result 'in_memory\\usade_inMem'>

In [10]:
arcpy.management.GetCount(fcInMem)

<Result '20198'>

In [13]:
with arcpy.da.SearchCursor(fc, ["AREA", "GEOID20"]) as cursor:
    for i in range(1000):
        for row in cursor:
            print(row)
            break

(2017.069300737762, 'USA_100030117003015')
(8752.48191427117, 'USA_100030127001011')
(16126.84501882378, 'USA_100030151002006')
(30234.23943813037, 'USA_100030121001003')
(35849.67236087531, 'USA_100030112051009')
(53003.08316452704, 'USA_100030148102015')
(13031.615290660204, 'USA_100030112062022')
(22459.04836752655, 'USA_100030168063009')
(70644.61767999936, 'USA_100030166141010')
(59969.73329627705, 'USA_100030164041013')
(14642.768994379925, 'USA_100030163081015')
(303946.21155354165, 'USA_100030169011014')
(32846.391443246735, 'USA_100030113002008')
(5048.982364435825, 'USA_100030136153012')
(16160.72551198477, 'USA_100030147052002')
(9874.675475018777, 'USA_100030150003019')
(4275.371549150043, 'USA_100030124003016')
(68188.13653097293, 'USA_100030147021047')
(6031578.21728173, 'USA_100030166081000')
(227780.66175818158, 'USA_100030166111001')
(105539.32917105955, 'USA_100030118002010')
(1571078.4224934704, 'USA_100030166101026')
(14227.344157861442, 'USA_100030144041023')
(1212